In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os

from urllib.request import urlopen

# 출력화면 초기화 하는 모듈
from IPython.display import clear_output

- 소스보기에서 데이터가 없었다.
- iframe도 없다.
- 비동기 통신인지 확인
    - crome에서 f12를 눌러 개발자 도구를 띄우고 네트워크 탭을 띄워준다.
  
- 변화를 주고 네트워크를 확인했을때 XHR이라고 나온다면 비동기 통신이다.
- 네트워크에 탭에 나와있는 주소를 더블클릭해서 브라우저에서 주조를 확인한다.

### 요청함수

In [3]:
def getSource(site):
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36',   
    }
    
    # 요청한다.
    response = requests.get(site, headers = header_info)
#     print(response.text)

    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text,'lxml')
    
    return soup

### 하루의 경기 결과를 가져오는 함수

In [93]:
def getData(soup):
    
    # 데이터를 담을 딕셔너리 생성
    data_dict ={
        '팀1' : [],
        '팀1_투수': [],
        '팀1_점수' : [],
        '팀2' : [],
        '팀2_투수' : [],
        '팀2_점수' : []
    }
    
    
    # 경기 결과 전체를 가져온다.
    a1 = soup.select_one('#todaySchedule')
    
    # li 클래스 end 태그들을 가져온다.
    a2 = a1.select('li.end')
    
    # li 태그의 수 만큼 반복
    for a3 in a2:
        # 경기 결과
        
        # 팀1 이름을 가져온다.
        a4 = a3.select_one('div.vs_lft > p > strong')
        data1 = a4.text.strip()
        
        # 팀1의 투수
        a5 = a3.select_one('div.vs_lft > p > span > a')
        if a5 != None:
            data2 = a5.text.strip()
        else:
            data2 = np.nan
        
        # 팀1 점수
        a6 = a3.select_one('div.vs_lft > strong')
        data3 = a6.text.strip()
        
        # 팀2 이름을 가져온다.
        a7 = a3.select_one('div.vs_rgt > p > strong')
        data4 = a7.text.strip()
       
        # 팀2 투수
        a8 = a3.select_one('div.vs_rgt > p > span > a')
        if a8 != None:
            data5 = a8.text.strip()
        else:
            data5 = np.nan
        
        # 팀2 점수
        a9 = a3.select_one('div.vs_rgt > strong')
        data6 = a9.text.strip()
        
        # 딕셔너리에 추가
        data_dict['팀1'].append(data1)
        data_dict['팀1_투수'].append(data2)
        data_dict['팀1_점수'].append(data3)
        data_dict['팀2'].append(data4)
        data_dict['팀2_투수'].append(data5)
        data_dict['팀2_점수'].append(data6)
        
        
    df1 = pd.DataFrame(data_dict)
    display(df1)
    
    # 데이터 프레임 저장
    if os.path.exists('baseball_2019.csv') == False:
        df1.to_csv('baseball_2019.csv', encoding='utf-8-sig', index=False)
    else:
        df1.to_csv('baseball_2019.csv', encoding='utf-8-sig', index=False, header=False, mode='a')
    
#         print("패배팀")
#         print(data1)
#         print(data2)
#         print(data3)
#         print("승리팀")
#         print(data4)
#         print(data5)
#         print(data6)
#         print('-'*20)

### 다음 날짜 값을 가지고 오는 함수

In [94]:
def getNext(soup):
    a1 = soup.select_one('body > div > ul > li:nth-child(6) > a')
    onclick = a1.attrs['onclick']
    data1 = onclick.split("'")
    data1 = data1[1]
    
    return data1

In [95]:
now = '20190312'


while True:
    time.sleep(1)
    clear_output(wait=True)
    print(f'{now} 데이터 수집중')
    site = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={now}&category=kbo'

    soup = getSource(site)
    now = getNext(soup)
    getData(soup)
    
    print(now)
    
    if now[:4] == '2020':
        print('2019년 데이터 수집완료')
        break;
    

20191026 데이터 수집중


,팀1,팀1_투수,팀1_점수,팀2,팀2_투수,팀2_점수
0,두산,이용찬,11,키움,브리검,9


20200421
2019년 데이터 수집완료
